In [ ]:
!pip install openai==0.28.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [ ]:
!git clone https://github.com/raphael-sch/VELMA.git
%cd VELMA

Cloning into 'VELMA'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 137 (delta 14), reused 24 (delta 10), pack-reused 104 (from 1)
Receiving objects: 100% (137/137), 101.75 MiB | 6.66 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Updating files: 100% (78/78), done.
/content/VELMA


In [ ]:
from PIL import Image
from io import BytesIO
import pandas as pd
import openai
import base64
import requests
import io
import math
import html
import re
import cv2
import numpy as np
import imageio
import os
import csv
import json

aca se guardan set de coordenadas, distancias y ruta a la carpeta donde estan las imagenes e instancia

borrar lo que tenga coordenadas de los cognitive maps json, contar cuantas rutas hay y el orden

In [ ]:
# Function to calculate heading between two points
def calculate_heading(lat1, lon1, lat2, lon2):
    """Calculate the heading between two geographic points."""
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    d_lon = lon2 - lon1
    x = math.sin(d_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(d_lon))

    initial_bearing = math.atan2(x, y)
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

In [ ]:
def location_heading_func(index, threshold, my_panoid):
    data = []
    for i, pan in enumerate(my_panoid[:-1]):  # Loop through all but the last pano
        current_location = nodes[nodes['Panoid'] == pan]
        next_location = nodes[nodes['Panoid'] == my_panoid[i + 1]]  # Use the next pano directly

        # Check if the current and next locations are found
        if not current_location.empty and not next_location.empty:
            current_point = current_location['location'].values[0]
            next_point = next_location['location'].values[0]

            # If the points are formatted as strings (e.g., "lat, lon"), split them
            if isinstance(current_point, str):
                current_point = [float(coord.strip()) for coord in current_point.split(',')]
            if isinstance(next_point, str):
                next_point = [float(coord.strip()) for coord in next_point.split(',')]

            # Calculate the heading
            t_heading = calculate_heading(current_point[0], current_point[1], next_point[0], next_point[1])
            # Append data to the list
            data.append({
                'Current Point': current_point,
                'Current Heading': t_heading
                })

    location_heading = pd.DataFrame(data)

    # List to store exceeding threshold data
    exceeding_threshold_data = []
    exceeding_threshold_data.append(location_heading.iloc[0].to_dict())
    # Iterate through the DataFrame to calculate heading differences
    for i in range(len(location_heading) - 1):
        current_heading = location_heading["Current Heading"].iloc[i] % 360
        next_heading = location_heading["Current Heading"].iloc[i + 1] % 360

        # Calculate the absolute difference in headings
        heading_difference = abs(current_heading - next_heading)

        # Normalize the heading difference to be within 0-180 degrees
        if heading_difference > 180:
            heading_difference = 360 - heading_difference

        # Check if the heading difference exceeds the threshold
        if heading_difference >= heading_threshold:
            current_point = location_heading["Current Point"].iloc[i]
            next_point = location_heading["Current Point"].iloc[i + 1]
            exceeding_threshold_data.append({
                'Current Point': current_point,
                'Current Heading': current_heading
            })
    exceeding_threshold_data.append({
        'Current Point': location_heading["Current Point"].iloc[-1],  # Last row's Current Point
        'Current Heading': location_heading["Current Heading"].iloc[-2]  # Second-to-last row's Current Heading
    })

    exceeding_threshold_df = pd.DataFrame(exceeding_threshold_data)
    location = exceeding_threshold_df['Current Point']
    heading = exceeding_threshold_df['Current Heading']

    return (location, heading)

In [ ]:
# Path to your JSON file
json_file = '/content/VELMA/datasets/map2seq_seen/data/train.json'

routes = []

# Load the JSON file and split the content by newlines
with open(json_file, 'r') as file:
    content = file.read()  # Read the file content

    # Split by lines (assuming each line is a separate JSON object)
    json_objects = content.strip().split('\n')

    # Parse each JSON object separately
    for obj in json_objects:
        try:
            data = json.loads(obj)  # Parse each JSON object
            routes.append(data)    # Append the parsed data to the list
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON object: {e}")

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(routes)

# Print the DataFrame to check if everything is loaded correctly
print(df)

# If you need to access the length of 'id' for each row in the DataFrame:
df['id_length'] = df['id'].astype(str).apply(len)

# Print the DataFrame with the new column
unique_id_count = df['instructions_id'].nunique()
print(f"Number of unique IDs: {unique_id_count}")

        id  instructions_id  \
0     3890             6199   
1      762             2386   
2      566             2170   
3     6621            11773   
4     3206             5181   
...    ...              ...   
6067  6941            11942   
6068  4245             6023   
6069  3130             5026   
6070  4233             5997   
6071  4241             6014   

                                          route_panoids  \
0     [pC21-8fg49EaRxpvXF2M_A, lXPHjkUOnMF4c4q_laK3t...   
1     [n11iwpgfziGtTxMiEfe2tA, fC-4WDDSswU95ul7VjqtK...   
2     [rYkV6xH5B9qBBrs53gDDsw, mzcnwsvM2XVAohfNtj863...   
3     [LxkBjn_U-VbZDPgCEoY4TQ, ai-OcfBxtL1RmQKe1s-9z...   
4     [eXCmBxpFyJvJCYKF9l1k9A, 5UmtlXcSDEXdfCFXcHPmz...   
...                                                 ...   
6067  [RzmE_OnMqrFKr0nIMiOC9g, ouvW2TDorMt6vuaA5b4gY...   
6068  [XlntDT1inp5L7EzgNhSbEg, dqTddv8tYDc2F0X9v5evj...   
6069  [gG5kygeBGUYtCZuSweJChw, bBQ-MVk4cgtBbB3lp8vzS...   
6070  [7bXzTG88LsoAY5slrW3-Kw, ix4zht

In [ ]:
# Path to your text file
file_path = '/content/VELMA/datasets/map2seq_seen/graph/nodes.txt'

# Define the column names
column_names = ['Panoid', 'Heading', 'Lat', 'Lon', 'type']

# Read the text file into a DataFrame without a header, and assign column names
nodes = pd.read_csv(file_path, header=None, names=column_names)
nodes['location'] = nodes['Lat'].astype(str) + ', ' + nodes['Lon'].astype(str)

nodes = nodes.drop(columns=['Lat','Lon'])
nodes.reset_index(drop=True, inplace=True)

In [ ]:
# Convert all non-serializable types to standard Python types
def convert_to_standard_types(data):
    if isinstance(data, dict):
        return {key: convert_to_standard_types(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_standard_types(item) for item in data]
    elif isinstance(data, np.integer):  # Handle numpy integer
        return int(data)
    elif isinstance(data, np.floating):  # Handle numpy float
        return float(data)
    else:
        return data

In [ ]:
# Function to calculate the Haversine distance between two points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Radius of the Earth in meters
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = (np.sin(dlat / 2) ** 2 +
         np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c  # Distance in meters
    return distance

# Function to interpolate points between two latitude-longitude coordinates
def interpolate_points(lat1, lon1, lat2, lon2, num_points):
    lat_points = np.linspace(lat1, lat2, num=num_points + 2)[1:-1]
    lon_points = np.linspace(lon1, lon2, num=num_points + 2)[1:-1]
    return list(zip(lat_points, lon_points))

# Function to convert numpy types to standard Python types
def convert_to_standard_types(obj):
    if isinstance(obj, dict):
        return {k: convert_to_standard_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_standard_types(i) for i in obj]
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

In [ ]:
# Iterator
#fixed =[2849]

#for i in fixed:
for i in range(0, 6072):
    Instance_id = df['id'][i]
    heading_threshold = 30
    my_panoid = df['route_panoids'][i]

    (locat, headi) = location_heading_func(index=i, threshold=heading_threshold, my_panoid=my_panoid)
    location = pd.DataFrame(locat)
    heading_data = []
    distance_data = []
    location_list = []  # Store locations with interpolated points included
    distance_data = []
    k = 0
    # Loop through all but the last pano
    for j in range(len(location) - 1):
        current_location = location.iloc[j]['Current Point']
        next_location = location.iloc[j + 1]['Current Point']

        current_point = current_location
        next_point = next_location

        # Calculate heading and distance between consecutive points
        point_heading = calculate_heading(current_point[0], current_point[1], next_point[0], next_point[1])
        point_distance = haversine(current_point[0], current_point[1], next_point[0], next_point[1])

        if point_distance > 10 or k==0:
            # Add the current location
            location_list.append({
                'lat': current_point[0],
                'lon': current_point[1],
                'heading': int(round(point_heading)),
                'file_name': f"{Instance_id}_{k}.jpg"
            })
            k += 1

    # Append the final point
    location_list.append({
        'lat': round(next_point[0], 8),
        'lon': round(next_point[1], 8),
        'heading': int(round(point_heading)),
        'file_name': f"{Instance_id}_{k}.jpg"
    })
    k += 1
    # Create dictionary for JSON output
    my_dictionary = {
        "Instance_id": Instance_id,
        "Locations": location_list,
    }

    # Define the filename for the JSON file
    filename = '/content/train_visual_environment.json'

    # Check if the file exists
    if os.path.exists(filename):
        try:
            with open(filename, 'r') as file:
                content = file.read().strip()
                accumulated_data = json.loads(content) if content else []
        except json.JSONDecodeError:
            print("Error decoding JSON. Initializing a new list.")
            accumulated_data = []
    else:
        accumulated_data = []

    # Append the new dictionary to the accumulated data
    accumulated_data.append(my_dictionary)

    # Convert accumulated_data to standard types and save to JSON
    with open(filename, 'w') as file:
        json.dump(convert_to_standard_types(accumulated_data), file, indent=4)

In [ ]:
from geopy.distance import geodesic

# Load the JSON file
file_path = filename
with open(file_path, 'r') as file:
    data = json.load(file)

def calculate_intermediate_points(location1, location2, num_points):
    """
    Calculate intermediate points between two locations.

    Args:
        location1 (tuple): Coordinates (lat, lon) of the first point.
        location2 (tuple): Coordinates (lat, lon) of the second point.
        num_points (int): Number of intermediate points to generate.

    Returns:
        list: A list of intermediate points [(lat, lon), ...].
    """
    lat1, lon1 = location1
    lat2, lon2 = location2
    intermediate_points = []
    for i in range(1, num_points + 1):
        fraction = i / (num_points + 1)
        lat = lat1 + fraction * (lat2 - lat1)
        lon = lon1 + fraction * (lon2 - lon1)
        intermediate_points.append((lat, lon))
    return intermediate_points

# Process each instance
updated_data = []
for instance in data:
    instance_id = instance["Instance_id"]
    locations = instance["Locations"]
    updated_locations = []

    for i in range(len(locations) - 1):
        loc1 = locations[i]
        loc2 = locations[i + 1]

        # Calculate the distance between consecutive points
        coord1 = (loc1["lat"], loc1["lon"])
        coord2 = (loc2["lat"], loc2["lon"])
        distance = geodesic(coord1, coord2).meters

        # Add the current location
        updated_locations.append(loc1)

        # Determine the number of intermediate points based on distance
        if 100 <= distance < 150:
            num_points = 1
        elif 150 <= distance < 200:
            num_points = 2
        elif distance >= 200:
            num_points = 3
        else:
            num_points = 0

        # Generate and add intermediate points
        if num_points > 0:
            intermediate_points = calculate_intermediate_points(coord1, coord2, num_points)
            heading = loc1["heading"]
            for idx, (lat, lon) in enumerate(intermediate_points, start=1):
                intermediate_location = {
                    "lat": round(lat,7),
                    "lon": round(lon,7),
                    "heading": heading,  # Use the heading from the first point
                    "file_name": f"{instance_id}_intermediate_{i}_{idx}.jpg"
                }
                updated_locations.append(intermediate_location)

    # Add the last location of the instance
    updated_locations.append(locations[-1])
    updated_data.append({
        "Instance_id": instance_id,
        "Locations": updated_locations
    })

# Save the updated data
output_path = filename
with open(output_path, 'w') as output_file:
    json.dump(updated_data, output_file, indent=4)

output_path

'/content/train_visual_environment.json'

In [ ]:
# Remove "file_name" from all locations in the updated data
for instance in updated_data:
    for location in instance["Locations"]:
        location.pop("file_name", None)  # Remove the "file_name" key if it exists

# Save the updated data without "file_name"
output_path_no_filename = filename
with open(output_path_no_filename, 'w') as output_file:
    json.dump(updated_data, output_file, indent=4)

output_path_no_filename


'/content/train_visual_environment.json'